In [2]:
from omero.gateway import BlitzGateway

In [14]:
conn = BlitzGateway("Asterix", "abc123",group='microscope_1', host="localhost", port=6064, secure=True)
conn.connect()

True

In [32]:
user = conn.getUser()
print("Current user:")
print("   ID:", user.getId())
print("   Username:", user.getName())
print("   Full Name:", user.getFullName())

# Check if you are an Administrator
print("   Is Admin:", conn.isAdmin())
if not conn.isFullAdmin():
    # If 'Restricted Administrator' show privileges
    print(conn.getCurrentAdminPrivileges())

print("Member of:")
for g in conn.getGroupsMemberOf():
    print("   ID:", g.getName(), " Name:", g.getId())
group = conn.getGroupFromContext()
print("Current group: ", group.getName())

# List the group owners and other members
owners, members = group.groupSummary()
print("   Group owners:")
for o in owners:
    print("     ID: %s %s Name: %s" % (
        o.getId(), o.getOmeName(), o.getFullName()))
print("   Group members:")
for m in members:
    print("     ID: %s %s Name: %s" % (
        m.getId(), m.getOmeName(), m.getFullName()))

print("Owner of:")
for g in conn.listOwnedGroups():
    print("   ID: ", g.getName(), " Name:", g.getId())

# Added in OMERO 5.0
print("Admins:")
for exp in conn.getAdministrators():
    print("   ID: %s %s Name: %s" % (
        exp.getId(), exp.getOmeName(), exp.getFullName()))

# The 'context' of our current session
ctx = conn.getEventContext()
# print(ctx)     # for more info

Current user:
   ID: 3
   Username: Asterix
   Full Name: Asterix CoreFacilityStaff
   Is Admin: False
[]
Member of:
   ID: core_facility_staff  Name: 4
   ID: microscope_1  Name: 5
   ID: microscope_2  Name: 6
Current group:  core_facility_staff
   Group owners:
     ID: 2 Abraracurcix Name: Abraracurcix CoreFacilityManager
   Group members:
     ID: 5 Panoramix Name: Panoramix CoreFacilityStaff
     ID: 3 Asterix Name: Asterix CoreFacilityStaff
     ID: 4 Obelix Name: Obelix CoreFacilityStaff
Owner of:
   ID:  microscope_1  Name: 5
   ID:  microscope_2  Name: 6
Admins:
   ID: 0 root Name: root root


In [4]:
for group in conn.listOwnedGroups():
    owners, members = group.groupSummary()
    print("   Group owners:")
    for o in owners:
        print("     ID: %s %s Name: %s" % (
            o.getId(), o.getOmeName(), o.getFullName()))
    print("   Group members:")
    for m in members:
        print("     ID: %s %s Name: %s" % (
            m.getId(), m.getOmeName(), m.getFullName()))

   Group owners:
     ID: 3 Asterix Name: Asterix CoreFacilityStaff
     ID: 4 Obelix Name: Obelix CoreFacilityStaff
   Group members:
     ID: 5 Panoramix Name: Panoramix CoreFacilityStaff
     ID: 2 Abraracurcix Name: Abraracurcix CoreFacilityManager
   Group owners:
     ID: 3 Asterix Name: Asterix CoreFacilityStaff
     ID: 5 Panoramix Name: Panoramix CoreFacilityStaff
   Group members:
     ID: 4 Obelix Name: Obelix CoreFacilityStaff
     ID: 2 Abraracurcix Name: Abraracurcix CoreFacilityManager


In [67]:
for i in conn.listOwnedGroups():
    print(f'------------------------------{i.getName()}----------------------------------')
    conn.SERVICE_OPTS.setOmeroGroup(i.getId())
    for project in conn.getObjects("Project"):
        print(project.getName(), project.getId())

------------------------------microscope_1----------------------------------
Field Illumination 60x 1
Field Illumination 10x 2
PSF beads 60x 3
------------------------------microscope_2----------------------------------
Field Illumination 40x 4
Field Illumination 20x 5
PSF beads 40x 6


In [28]:
g = conn.listProjects()

In [47]:
for project in conn.getObjects("Project", opts={'group': 4}):
    print(project.getName(), project.getId())

Field Illumination 60x 1
Field Illumination 10x 2
Field Illumination 40x 4
Field Illumination 20x 5
PSF beads 60x 3
PSF beads 40x 6


In [ ]:
def get_data_ids(conn):
    for i in conn.listOwnedGroups():
        conn.SERVICE_OPTS.setOmeroGroup(i.getId())
        for project in conn.getObjects("Project"):
            print(project.getName(), project.getId())

In [68]:
conn.SERVICE_OPTS.setOmeroGroup(-1)
for image in conn.getObjects("Image"):
    print(image.getName(), image.getId())

FI_60x_DAPI_2021-02-01 1
FI_60x_FITC_2021-02-01 2
FI_60x_TRITC_CY5_2021-02-01 3
FI_60x_DAPI_2021-03-01 4
FI_60x_FITC_2021-03-01 5
FI_60x_TRITC_CY5_2021-03-01 6
FI_10x_DAPI_2021-03-04 7
FI_10x_FITC_2021-03-04 8
FI_10x_TRITC_CY5_2021-03-04 9
FI_10x_DAPI_2021-05-04 10
FI_10x_FITC_2021-05-04 11
FI_10x_TRITC_CY5_2021-05-04 12
FI_10x_DAPI_2021-07-04 13
FI_10x_FITC_2021-07-04 14
FI_10x_TRITC_CY5_2021-07-04 15
FI_10x_DAPI_2021-09-04 16
FI_10x_FITC_2021-09-04 17
FI_10x_TRITC_CY5_2021-09-04 18
FI_60x_DAPI_2021-02-01 21
FI_60x_FITC_2021-02-01 22
FI_60x_TRITC_CY5_2021-02-01 23
FI_60x_DAPI_2021-03-01 24
FI_60x_FITC_2021-03-01 25
FI_60x_TRITC_CY5_2021-03-01 26
FI_10x_DAPI_2021-03-04 27
FI_10x_FITC_2021-03-04 28
FI_10x_TRITC_CY5_2021-03-04 29
FI_10x_DAPI_2021-05-04 30
FI_10x_FITC_2021-05-04 31
FI_10x_TRITC_CY5_2021-05-04 32
FI_10x_DAPI_2021-07-04 33
FI_10x_FITC_2021-07-04 34
FI_10x_TRITC_CY5_2021-07-04 35
FI_10x_DAPI_2021-09-04 36
FI_10x_FITC_2021-09-04 37
FI_10x_TRITC_CY5_2021-09-04 38
PSF-beads_60x

In [25]:
dataset_4 = conn.getObject("Dataset", 4)
dataset_6 = conn.getObject("Dataset", 6)


In [28]:
dataset_4.getAnnotation()

<MapAnnotationWrapper id=6>

In [34]:
dataset_4.getAnnotationCounts()

{'TagAnnotation': 0,
 'FileAnnotation': 0,
 'CommentAnnotation': 0,
 'LongAnnotation': 0,
 'MapAnnotation': 2,
 'OtherAnnotation': 0}

In [35]:
an= conn.getObject("MapAnnotation", 6)


In [33]:
an.GET

'microscope-metrics parameters for the analysis on the dataset'

In [36]:
ann = dataset_4.getAnnotation()
print("Found Annotation with namespace: ", ann.getNs())

Found Annotation with namespace:  https://w3id.org/MontpellierRessourcesImagerie/microscopemetrics-schema/FieldIlluminationInput


In [40]:
ann = dataset_4.getAnnotation()
print("Found Annotation with namespace: ", ann.getNs())

Found Annotation with namespace:  https://w3id.org/MontpellierRessourcesImagerie/microscopemetrics-schema/FieldIlluminationInput


In [62]:
def get_dataset_ids_lists(conn, project_id):
    """
    Get the processed and unprocessed dataset ids for a project
    """
    processed_datasets = {}
    unprocessed_datasets = {}
    project = conn.getObject("Project", project_id)
    for dataset in project.listChildren():
        try:
            dataset.getAnnotation().getNs()
            processed_datasets.update(dataset.getId())
        except AttributeError:
            unprocessed_datasets.update({'ID': dataset.getId())
    return processed_datasets, unprocessed_datasets

In [63]:
l, n = get_dataset_ids_lists(conn, 3)

TypeError: 'int' object is not iterable

In [60]:
l

{'ID': 5}

In [61]:
n

{}